In [1]:
import sys
sys.path.insert(0, '../../src')

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.special import comb


from LDPC import LDPC
from McEliece import McEliece
from utils import gaussjordan

In [3]:

# EXAMPLE USAGE

n = 300
d_v = 6
d_c = 10

ldpc = LDPC.from_params(n, d_v, d_c)

word = np.random.randint(2, size=ldpc.getG().shape[0])
print("word:", word)

crypto = McEliece.from_linear_code(ldpc, 3)

encrypted = crypto.encrypt(word)
print("encrypted:", encrypted)

decrypted = crypto.decrypt(encrypted)
print("decrypted:", decrypted)

assert (word == decrypted).all()

word: [0 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0
 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 1 0 0 1 0
 0 0 0 0 1 0 1 1 0 0 0 0 0 1]
encrypted: [1 1 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 0 1 0 0
 0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0
 0 1 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 1 0
 1 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1
 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0
 1 0 1 1 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 0 1 0 0 1 1 0 1 1
 0 0 1 0 1 1 1 0 1 1 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 1 1
 1 0 1 0]
decrypted: [0 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1

In [35]:
def information_set_decoding(encrypted, public_key, epochs):
    rng = np.random.default_rng()
    G, t = public_key
    k, n = G.shape
    #print("G", G.shape)
    arr = np.arange(n)
    wt = -1
    step = 0
    tao = comb(n, t) / comb(n - k, t)
    #print("tao", tao)
    E = np.eye(k, dtype=int)
    while wt != t and step <= max(2*tao, epochs):
        #print("epoch")
        rng.shuffle(arr)
        J = arr[:k] #random subset
        G1 = G[:, J]
        #print(G1.shape)
        I, G1_inv = gaussjordan(G1, True)
        if not (I == E).all():
            #print("not inv")
            continue
        #print(G1_inv.shape)
        wt = np.sum((encrypted + encrypted[J] @ G1_inv @ G) % 2)
        step += 1
        #print(step)
        
    if wt == t:
        #print((encrypted[J] @ G1_inv) % 2)
        return (encrypted[J] @ G1_inv) % 2, step, tao
    
    return -1, -1, -1

In [5]:
public_key = crypto.public_key

In [11]:
decoded, step, tao = information_set_decoding(encrypted, public_key, 100)

In [12]:
decoded

array([0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1])

In [13]:
len(decoded)

125

In [14]:
decrypted

array([0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1])

In [15]:
assert(decrypted == decoded).all()

In [28]:
n = 1000
d_v = 8
d_c = 10

ldpc = LDPC.from_params(n, d_v, d_c)

print(ldpc.guess_code_max_error())

55


In [16]:
n = 1000
d_v = 8
d_c = 10

In [18]:
t = 0
result_time_of_operations = []
result_steps = []
result_taos = []

for t in range(31):
    
    time_of_operation = []
    steps = []
    taos = []
    
    for i in range(10):
        
        ldpc = LDPC.from_params(n, d_v, d_c)

        word = np.random.randint(2, size=ldpc.getG().shape[0])

        crypto = McEliece.from_linear_code(ldpc, t)

        encrypted = crypto.encrypt(word)

        decrypted = crypto.decrypt(encrypted)

        assert (word == decrypted).all()

        public_key = crypto.public_key

        start = time.time()
        decoded, step, tao = information_set_decoding(encrypted, public_key, 100)
        assert(decrypted == decoded).all()
        end = time.time()

        time_of_operation.append(end - start)
        steps.append(step)
        taos.append(tao)
        
    time_of_operation = np.asarray(time_of_operation)
    steps = np.asarray(steps)
    taos = np.asarray(taos)

    result_time_of_operations.append(np.mean(time_of_operation))
    result_steps.append(np.mean(steps))
    result_taos.append(np.mean(taos))
    print(t)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13


TypeError: 'NoneType' object is not iterable